In [1]:
from Bio import SeqIO
from Bio.Seq import Seq
import pandas

In [2]:
genome = ("/Users/carolinaatria/Desktop/ADesktop/Studium/Master/master_thesis/sponge/data_from_roger/python/data.nosync/genome_assembly/SDO_hapA_v0.1.fasta")
gene_annotations = ("/Users/carolinaatria/Desktop/ADesktop/Studium/Master/master_thesis/sponge/data_from_roger/python/data.nosync/isoseq_transcriptome_annotation/corr_final.gtf")
atac_peaks = ("/Users/carolinaatria/Desktop/ADesktop/Studium/Master/master_thesis/sponge/data_from_roger/python/data.nosync/Sdom_Isoseq_222253/atac_peak_annotation.tsv")
genome_bed_file = "/Users/carolinaatria/Desktop/ADesktop/Studium/Master/master_thesis/sponge/data_from_roger/python/data.nosync/Sdom_Isoseq_222252/cluster_analysis/all_tss.bed"
gtf_transcriptome = "/Users/carolinaatria/Desktop/ADesktop/Studium/Master/master_thesis/sponge/data_from_roger/python/data.nosync/isoseq_transcriptome_annotation/corr_final.gtf"

In [3]:
genome = SeqIO.to_dict(SeqIO.parse(genome, "fasta"))
tss_data,gene_list = [],[]
with open(genome_bed_file, "r") as file:
    for line in file:
        chromosome, start, end, gene_name = line.strip().split("\t")
        tss_data.append((chromosome, int(start), int(end), gene_name))
        gene_list.append(gene_name)

In [4]:
tss_data = pandas.DataFrame(tss_data, columns=["chrom", "start", "end", "gene"])

In [5]:
tss_data

,chrom,start,end,gene
0,sca1,63074,63094,SUB2.g1
1,sca1,96869,96889,SUB2.g10
2,sca1,534938,534958,SUB2.g100
3,sca2,206841,206861,SUB2.g1000
4,sca1,903690,903710,SUB2.g10000
...,...,...,...,...
13139,sca1,863137,863157,SUB2.g9995
13140,sca1,865362,865382,SUB2.g9996
13141,sca1,867482,867502,SUB2.g9997
13142,sca1,899859,899879,SUB2.g9998


In [6]:
genes = pandas.read_csv(gtf_transcriptome, sep="\t", header=None)

In [7]:
peaks = pandas.read_csv(atac_peaks,sep='\t')

peaks

,chrom,start,end,gene,distance,peak_type,TEST
0,sca1,60053,60744,SUB2.g9957,0,promoter,NaN
1,sca1,62657,63530,SUB2.g1,0,promoter,NaN
2,sca1,62657,63530,SUB2.g2,0,promoter,NaN
3,sca1,101067,101865,SUB2.g11,0,promoter,NaN
4,sca1,128580,129401,SUB2.g9974,0,promoter,NaN
...,...,...,...,...,...,...,...
10652,sca763,1999,2897,SUB2.g9930,0,distal,NaN
10653,sca763,1999,2897,SUB2.g9931,0,promoter,NaN
10654,sca766,4348,5294,SUB2.g13134,0,promoter,NaN
10655,sca766,4348,5294,SUB2.g9935,0,promoter,NaN


In [9]:
#count how many times "promoter" appears in the "peak_type" column
peaks["peak_type"].value_counts()

#create new table with only the entries for "promoter" in "peak_type" column
# promoters = peaks[peaks["peak_type"] == "promoter"]

promoter    6766
distal      3891
Name: peak_type, dtype: int64

In [10]:
promoters

,chrom,start,end,gene,distance,peak_type,TEST
0,sca1,60053,60744,SUB2.g9957,0,promoter,NaN
1,sca1,62657,63530,SUB2.g1,0,promoter,NaN
2,sca1,62657,63530,SUB2.g2,0,promoter,NaN
3,sca1,101067,101865,SUB2.g11,0,promoter,NaN
4,sca1,128580,129401,SUB2.g9974,0,promoter,NaN
...,...,...,...,...,...,...,...
10651,sca753,12072,12903,SUB2.g9922,0,promoter,NaN
10653,sca763,1999,2897,SUB2.g9931,0,promoter,NaN
10654,sca766,4348,5294,SUB2.g13134,0,promoter,NaN
10655,sca766,4348,5294,SUB2.g9935,0,promoter,NaN


In [162]:
df = peaks
# Initialize a set to store genes with shared peaks
genes_with_shared_peaks = set()
bidirectional_genes = []

# Iterate over rows in ATAC peaks dataframe
for index, row in peaks.iterrows():
    peak_start = row['start']
    peak_end = row['end']
    peak_gene = row['gene']
    peak_type = row["peak_type"]
    
    # Check if any gene in tss_data shares coordinates with the peak
    shared_genes = df[(df['chrom'] == row['chrom']) &
                         (peak_gene != df["gene"]) & # Exclude the gene itself
                            ((df['start'] <= peak_start) & (peak_start <= df['end']) |
                             (df['start'] <= peak_end) & (peak_end <= df['end']))]
    
    # Add genes with shared peaks to the set
    for _, gene_row in shared_genes.iterrows():
        genes_with_shared_peaks.add(gene_row['gene'])
        bidirectional_genes.append({"gene1":peak_gene,"gene2":shared_genes["gene"],"type":peak_type})


bidirectional_peaks = pandas.DataFrame(bidirectional_genes)


# Calculate the proportion of genes with shared peaks
total_genes_with_peaks = len(df['gene'].unique())
print("number of peaks that are shared",len(genes_with_shared_peaks))


proportion_shared_peaks = len(genes_with_shared_peaks) / total_genes_with_peaks

proportion_shared_peaks = len(genes_with_shared_peaks) / len(gene_list)

print("Percentage of peaks that share their peak with another gene:", round(proportion_shared_peaks * 100,2), "%")

proportion_shared_peaks = len(genes_with_shared_peaks) / total_genes_with_peaks

print("Percentage of genes that share their peak with another gene:", round(proportion_shared_peaks * 100,2), "%")

number of peaks that are shared 5194
Percentage of peaks that share their peak with another gene: 39.52 %
Percentage of genes that share their peak with another gene: 82.34 %


In [163]:
len(gene_list)

13144

In [160]:
df = promoters
# Initialize a set to store genes with shared peaks
genes_with_shared_peaks = set()
bidirectional_genes = []

# Iterate over rows in ATAC peaks dataframe
for index, row in peaks.iterrows():
    peak_start = row['start']
    peak_end = row['end']
    peak_gene = row['gene']
    peak_type = row["peak_type"]
    
    # Check if any gene in tss_data shares coordinates with the peak
    shared_genes = df[(df['chrom'] == row['chrom']) &
                         (peak_gene != df["gene"]) & # Exclude the gene itself
                            ((df['start'] <= peak_start) & (peak_start <= df['end']) |
                             (df['start'] <= peak_end) & (peak_end <= df['end']))]
    
    # Add genes with shared peaks to the set
    for _, gene_row in shared_genes.iterrows():
        genes_with_shared_peaks.add(gene_row['gene'])
        bidirectional_genes.append({"gene1":peak_gene,"gene2":shared_genes["gene"],"type":peak_type})

bidirectional_peaks = pandas.DataFrame(bidirectional_genes)

# Calculate the proportion of genes with shared peaks
total_genes_with_peaks = len(df['gene'].unique())
proportion_shared_peaks = len(genes_with_shared_peaks) / total_genes_with_peaks

print("number of peaks that are shared",len(genes_with_shared_peaks))

proportion_shared_peaks = len(genes_with_shared_peaks) / len(peaks["gene"].unique())

print("Percentage of peaks that share their peak with another gene:", round(proportion_shared_peaks * 100,2), "%")

proportion_shared_peaks = len(genes_with_shared_peaks) / total_genes_with_peaks

print("Percentage of genes that share their peak with another gene:", round(proportion_shared_peaks * 100,2), "%")

number of peaks that are shared {'SUB2.g3310', 'SUB2.g3972', 'SUB2.g3251', 'SUB2.g11627', 'SUB2.g6432', 'SUB2.g7227', 'SUB2.g9535', 'SUB2.g4789', 'SUB2.g3639', 'SUB2.g130', 'SUB2.g7218', 'SUB2.g7668', 'SUB2.g5304', 'SUB2.g12478', 'SUB2.g7091', 'SUB2.g6827', 'SUB2.g5864', 'SUB2.g7467', 'SUB2.g6471', 'SUB2.g1398', 'SUB2.g8610', 'SUB2.g12597', 'SUB2.g1551', 'SUB2.g1290', 'SUB2.g817', 'SUB2.g9278', 'SUB2.g980', 'SUB2.g7980', 'SUB2.g7138', 'SUB2.g4966', 'SUB2.g4948', 'SUB2.g12676', 'SUB2.g5521', 'SUB2.g3794', 'SUB2.g8149', 'SUB2.g10966', 'SUB2.g4226', 'SUB2.g6274', 'SUB2.g4158', 'SUB2.g11267', 'SUB2.g2820', 'SUB2.g7054', 'SUB2.g6075', 'SUB2.g6231', 'SUB2.g1334', 'SUB2.g5603', 'SUB2.g339', 'SUB2.g4223', 'SUB2.g2734', 'SUB2.g5245', 'SUB2.g9099', 'SUB2.g6797', 'SUB2.g12780', 'SUB2.g3523', 'SUB2.g9372', 'SUB2.g596', 'SUB2.g8170', 'SUB2.g10648', 'SUB2.g9433', 'SUB2.g5408', 'SUB2.g5887', 'SUB2.g10594', 'SUB2.g7214', 'SUB2.g3409', 'SUB2.g1999', 'SUB2.g10450', 'SUB2.g7304', 'SUB2.g56', 'SUB2.g7398'

In [161]:
len(genes_with_shared_peaks)

4296

In [159]:
#how many genes have any peaks
len(promoters["gene"].unique())

5211

In [156]:
peaks

,chrom,start,end,gene,distance,peak_type,TEST
0,sca1,60053,60744,SUB2.g9957,0,promoter,NaN
1,sca1,62657,63530,SUB2.g1,0,promoter,NaN
2,sca1,62657,63530,SUB2.g2,0,promoter,NaN
3,sca1,101067,101865,SUB2.g11,0,promoter,NaN
4,sca1,128580,129401,SUB2.g9974,0,promoter,NaN
...,...,...,...,...,...,...,...
10652,sca763,1999,2897,SUB2.g9930,0,distal,NaN
10653,sca763,1999,2897,SUB2.g9931,0,promoter,NaN
10654,sca766,4348,5294,SUB2.g13134,0,promoter,NaN
10655,sca766,4348,5294,SUB2.g9935,0,promoter,NaN
